<a href="https://colab.research.google.com/github/Jaybhatt216/EIT/blob/main/LSTM_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch 

## Build layer class
we will build a layer class on top of super class torch.nn.Module 

In [23]:
class Layer(torch.nn.Module):
  #size in is size before size out is out for this layer
  #nn parameter is how you feed the neural net
  def __init__(self,size_in,size_out,activation):
    super(Layer,self).__init__()
    self.weights = torch.nn.Parameter(torch.randn(size_in,size_out,requires_grad=True))
    self.bias = torch.nn.Parameter(torch.randn(1,size_out,requires_grad=True))
    self.activation = activation


  def Forward(self, z_in):
    return self.activation(z_in @self.weights + self.bias) 



# Layer initalize

In [24]:
forget = Layer(38,15,torch.nn.Sigmoid())
loss_func = torch.nn.MSELoss()
opt = torch.optim.Adam(forget.parameters())

In [25]:
x_in = torch.randn(1,38)
y = torch.rand(1,15)
print(x_in)
print(y)

tensor([[ 0.8628, -0.2809,  0.6315,  0.4143,  0.0128,  0.6250, -1.7685, -0.4364,
         -1.2858,  0.9515,  0.2903,  0.4718,  2.3356, -0.3511,  0.6788,  1.4555,
         -0.4935, -0.8775,  0.7540,  0.2212,  0.9125, -0.8637, -0.6678,  0.8865,
          0.9926, -0.5239, -0.7671, -1.2793,  0.3747,  0.6412, -0.3202, -0.0684,
          1.0640,  1.1321,  0.0516, -1.6982,  0.5919, -0.6233]])
tensor([[0.3317, 0.5169, 0.2276, 0.0018, 0.2405, 0.3352, 0.8150, 0.4871, 0.7911,
         0.3639, 0.6621, 0.7528, 0.0642, 0.0076, 0.3080]])


In [27]:
print(forget.bias)
out = forget.Forward(x_in)
loss = loss_func(out,y)
loss.backward()
opt.step()
opt.zero_grad()
print(forget.bias)

Parameter containing:
tensor([[-1.7523,  0.9407, -0.7421,  1.6027,  0.6439, -1.1389,  0.7855,  0.8710,
         -0.6491,  0.0872,  0.5905, -1.3373, -1.3933, -0.2294,  0.2139]],
       requires_grad=True)
Parameter containing:
tensor([[-1.7513,  0.9407, -0.7411,  1.6017,  0.6429, -1.1399,  0.7845,  0.8720,
         -0.6481,  0.0862,  0.5915, -1.3383, -1.3923, -0.2304,  0.2129]],
       requires_grad=True)


# RNN recurrent neural network
retains some info frome every stage

In [28]:
class RNN(torch.nn.Module):
  #size of memory is amount of data you will save layer to layer
  #only have 2 hidden layers memory and out
  def __init__(self,size_in,size_out,size_memory):
    super(RNN,self).__init__()
    self.size_memory = size_memory
    self.size_memory_layer = Layer(size_in + size_memory, size_memory, torch.tanh) #previous layers plus new data
    self.out_layer = Layer(size_memory, size_out, torch.sigmoid)

  def Forward(self,X):
    memory = torch.zeros(1,self.size_memory)
    y_hat = []
    for i in range(X.shape[0]):
      x_in = X[[i],:]
      z_in = torch.cat([x_in,memory], dim=1)
      memory = self.size_memory_layer.Forward(z_in)
      y_hat.append(self.out_layer.Forward(memory))

    return torch.cat(y_hat,dim=0)




In [30]:
rnn = RNN(38,15,5)
loss_func = torch.nn.MSELoss()
opt = torch.optim.Adam(rnn.parameters())


In [31]:
print(rnn.size_memory_layer.bias)
y_hat = rnn.Forward(x_in)
loss = loss_func(y_hat,y)
loss.backward()
opt.step()
opt.zero_grad
print()
print(rnn.size_memory_layer.bias)

Parameter containing:
tensor([[ 0.5963, -0.1294,  0.4803,  0.3810, -0.9185]], requires_grad=True)

Parameter containing:
tensor([[ 0.5973, -0.1284,  0.4794,  0.3820, -0.9185]], requires_grad=True)


# Build LSTM: Long Short Term Memory 

In [32]:
class LSTM(torch.nn.Module):
  #2 memory size long and short
  def __init__(self, size_in, size_out, size_long, size_short):
    super(LSTM, self).__init__()
    self.size_long = size_long
    self.size_short = size_short

    size_z = (size_in + size_short)

    #build series of layers

    #forget_gate layer
    self.forget_gate = Layer(size_z, size_long, torch.sigmoid)

    #memory gate layer
    self.memory_gate = Layer(size_z, size_long, torch.sigmoid)

    #memory layer
    self.memory_layer = Layer(size_z, size_long, torch.tanh)

    #recall gate 
    self.recall_gate = Layer(size_z, size_short, torch.sigmoid)
    #recall layer
    self.recall_layer = Layer(size_long, size_short, torch.tanh)

    #output layer
    self.output_gate = Layer(size_short, size_out, torch.sigmoid)

  def Forward(self, X):
    mem_short = torch.zeros(1,self.size_short)
    mem_long =  torch.zeros(1,self.size_long)
    y_hat = []
    for i in range(X.shape[0]): #for all the rows
      X_t = X[[i],:]
      z_t = torch.cat([X_t,mem_short], dim=1)
      mem_long = mem_long*(self.forget_gate.Forward(z_t))
      mem_long = mem_long+(self.memory_gate.Forward(z_t)*self.memory_layer.Forward(z_t))
      mem_short = self.recall_gate.Forward(z_t)+self.recall_layer.Forward(mem_long)


      y_hat.append(self.output_gate.Forward(mem_short))

    return torch.cat(y_hat, dim=0)

  def Generate(self, start,stop,random_factor):
    y_hat = [start]
    mem_long = torch.randn(1, self.size_long, random_factor)
    mem_short = torch.randn(1, self.size_short, random_factor)

    while (y_hat[-1] != stop).any() and len(y_hat)<30:
      x_t = y_hat[-1]
      z_t =  torch.cat([x_t, mem_short], dim=1)
      mem_long = mem_long * self.forget_gate.Forward(z_t)
      mem_long = mem_long + (self.memory_gate.Forward(z_t)*self.memory_layer.Forward(z_t))
      mem_short = self.recall_gate.Forward(z_t)+self.recall_layer.Forward(mem_long)
      out = self.output_gate.Forward(mem_short)
      out = torch.argmax(out, dim=1)
      y_hat.append(torch.zeros(stop.shape))
      y_hat[-1][0,out]= 1
    return torch.cat(y_hat,dim=0)






